In [ ]:
import cv2
import numpy as np
import imageio
import dlib
from IPython.display import Image
from google.colab.patches import cv2_imshow

In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

### **Function: `get_landmarks(image)`**  
#### **Purpose:**  
Detects facial landmarks for a given image using dlib's **face detector** and **shape predictor**.  

#### **Key Features:**  
- Returns **tie points (landmarks)** for the image, which include:  
  - **68 facial landmark points** (e.g., eyes, nose, mouth, jawline) deteted by **dlib's frontal face detector**.  
  - **4 corner points** of the image (top-left, top-right, bottom-left, bottom-right) added **manually**.   


In [ ]:
# Load the pre-trained dlib face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  # Model file from step 1

def get_landmarks(image):
    '''
    Function to generate tie points for an image using frontal face detector of dlib library which detects
    68 facial features
    '''

    # Detect faces in the image
    faces = detector(image)

    if len(faces) == 0:
        return []  # Return an empty list if no face is detected

    face = faces[0]  # Select the first detected face

    # 68 lanmark points
    shape = predictor(image, face)
    landmarks= [(shape.part(i).x, shape.part(i).y) for i in range(68)]  # 68 points

    # corner points
    landmarks.append((0, 0))
    landmarks.append((image.shape[1] - 1, 0))
    landmarks.append((0, image.shape[0] - 1))
    landmarks.append((image.shape[1] - 1, image.shape[0] - 1))

    return landmarks


### **Function: `delaunay_triangulation(img1, points1)`**  
#### **Purpose:**  
Performs **Delaunay triangulation** on a given set of landmark points and returns a list of triangle indices.  

#### **Key Steps:**  
1. **Initialize Subdiv2D** with the image dimensions for triangulation.  
2. **Insert landmark points** into the subdivision.  
3. **Retrieve triangles** as coordinate sets from `subdiv.getTriangleList()`.  
4. **Map each landmark point to its index** using a dictionary.  
5. **Filter valid triangles** (within image boundaries) and convert their vertices into indices.  
6. **Return a list of triangle indices**, where each triangle is represented by three landmark indices.  



In [ ]:
def delaunay_triangulation(img1, points1):

    # Create a new Subdiv2D object using the bounding rectangle
    subdiv = cv2.Subdiv2D((0, 0, img1.shape[1], img1.shape[0]))

    # subdiv.insert(points1)
    for pt in points1:
      subdiv.insert(pt)

    triangle_list = subdiv.getTriangleList()  # Get all triangles
    h, w = img1.shape[:2]  # Get image dimensions

    # Create a dictionary that maps each landmark point to its index in the points list
    point_index_dict = {tuple(point): index for index, point in enumerate(points1)}

    triangle_indices = []

    for t in triangle_list:
        pt1, pt2, pt3 = (t[0], t[1]), (t[2], t[3]), (t[4], t[5])  # Extract triangle vertices

        # Check if the points are inside the image boundaries
        if (0 <= pt1[0] < w and 0 <= pt1[1] < h and
            0 <= pt2[0] < w and 0 <= pt2[1] < h and
            0 <= pt3[0] < w and 0 <= pt3[1] < h):

            pt1, pt2, pt3 = (int(t[0]), int(t[1])), (int(t[2]), int(t[3])), (int(t[4]), int(t[5]))  # Convert to integer

            point_indices = (
                point_index_dict.get((t[0], t[1])),
                point_index_dict.get((t[2], t[3])),
                point_index_dict.get((t[4], t[5])),
            )
            if None not in point_indices:
                triangle_indices.append(point_indices)

    return triangle_indices


### **Function: `compute_morphed_triangle(img1, img2, triangle1, triangle2, triangle, alpha, result)`**
#### **Purpose:**  
Blends corresponding triangles from two images using **affine transformations** and **alpha blending** to create a smooth morphing effect.

#### **Key Steps:**  
1. **Extract Triangles:** Crop triangle regions from both images using bounding rectangles.  
2. **Create Masks:** Generate binary masks to isolate the triangles.  
3. **Affine Transformation:** Warp both triangles to match the target morphed triangle.  
4. **Blend Triangles:** Use **alpha blending** to combine them smoothly.  
5. **Merge into Output:** Insert the morphed triangle into the final result.  

#### **Output:**  
A seamlessly blended triangular region, contributing to a **smooth face morph**.


In [ ]:
def compute_morphed_triangle(img1, img2, triangle1, triangle2, morphed_triangle, alpha, result):
    """
    triangle1: Contains the (x, y) coordinates of the vertices of triangle1 in img1
    triangle2: Contains the (x, y) coordinates of the vertices of triangle2 in img2
    morphed_triangle: Contains the (x, y) coordinates of the vertices of the morphed triangle in the morphed image
    """
    # Calculate the bounding rectangle for triangle1 and crop the triangle
    rect1 = cv2.boundingRect(triangle1)
    x1, y1, w1, h1 = rect1
    cropped_triangle1 = img1[y1 : y1 + h1, x1 : x1 + w1]

    # Calculate the bounding rectangle for triangle2 and crop the triangle
    rect2 = cv2.boundingRect(triangle2)
    x2, y2, w2, h2 = rect2
    cropped_triangle2 = img2[y2 : y2 + h2, x2 : x2 + w2]

    # Calculate the bounding rectangle for the morphed triangle and create a mask for it
    r = cv2.boundingRect(morphed_triangle)
    x, y, w, h = r

    # Offset the points in the triangle so they are relative to the bounding rectangle
    t1_offset = np.array(triangle1 - [x1, y1], np.int32)
    t2_offset = np.array(triangle2 - [x2, y2], np.int32)
    t_offset = np.array(morphed_triangle - [x, y], np.int32)

    # Create a mask for the cropped triangle(single-channel, binary)
    mask1 = np.zeros((h1, w1), np.uint8)
    mask2 = np.zeros((h2, w2), np.uint8)
    mask = np.zeros((h, w), np.uint8)

    cv2.fillConvexPoly(mask1, t1_offset, (1.0, 1.0, 1.0), 16, 0)
    cv2.fillConvexPoly(mask2, t2_offset, (1.0, 1.0, 1.0), 16, 0)
    cv2.fillConvexPoly(mask, t_offset, (1.0, 1.0, 1.0), 16, 0)

    # Apply the mask to the cropped triangle1
    cropped_triangle1 = cv2.bitwise_and(cropped_triangle1, cropped_triangle1, mask=mask1)

    # Apply the mask to the cropped triangle2
    cropped_triangle2 = cv2.bitwise_and(cropped_triangle2, cropped_triangle2, mask=mask2)

    # Transform triangle1 and triangle2 to the morphed triangle
    t1_offset = np.float32(t1_offset)
    t2_offset = np.float32(t2_offset)
    t_offset = np.float32(t_offset)

    # Calculate the affine transformation matrices for the first and second triangles
    M1 = cv2.getAffineTransform(t1_offset, t_offset)
    wrap_triangle1 = cv2.warpAffine(cropped_triangle1, M1, (w, h))

    M2 = cv2.getAffineTransform(t2_offset, t_offset)
    wrap_triangle2 = cv2.warpAffine(cropped_triangle2, M2, (w, h))

    # Blend triangles
    morphed_triangle = cv2.addWeighted(wrap_triangle1, 1 - alpha, wrap_triangle2, alpha, 0)

    # Add the morphed triangle to the new image using the mask

    for c in range(3):  # Loop over all three channels (RGB)
        result[y:y+h, x:x+w, c] = ((1.0) - mask) * result[y:y+h, x:x+w, c] + morphed_triangle[:, :, c] * mask


### **Function: `generate_morphed_points(points1, points2, alpha)`**  
#### **Purpose:**  
Computes **intermediate landmark positions** between two sets of points using **linear interpolation**.  

#### **Key Steps:**  
1. Iterate through each pair of corresponding points in `points1` and `points2`.  
2. Compute **interpolated coordinates** using:  
   $$ x = (1 - \alpha) \cdot x_1 + \alpha \cdot x_2 $$  
   $$ y = (1 - \alpha) \cdot y_1 + \alpha \cdot y_2 $$  
3. Return the list of **morphed points**.  

#### **Output:**  
A **list of tuples** representing the **interpolated landmark positions** for the given `alpha` value.  

In [ ]:
def generate_morphed_points(points1, points2, alpha):
    """
    Computes intermediate landmark positions using linear interpolation.
    """
    morphed_points = [((1 - alpha) * p1[0] + alpha * p2[0],
                      (1 - alpha) * p1[1] + alpha * p2[1])
                      for p1, p2 in zip(points1, points2)]

    return morphed_points

### **Function: `generate_morphed_frame(img1, img2, points1, points2, morphed_points, triangle_indices, alpha)`**  
#### **Purpose:**  
Generates a **single morphed frame** by warping and blending **triangular regions** between two images.  

#### **Key Steps:**  
1. **Initialize an empty image** to store the morphed frame.  
2. Iterate over each **Delaunay triangle** index in `triangle_indices`:  
   - Extract **corresponding triangles** from `img1`, `img2`, and the **interpolated** frame.  
   - **Warp and blend** the triangles into the morphed image using `compute_morphed_triangle()`.  
3. Apply **median filtering** (`cv2.medianBlur()`) to **smooth the output**.  
4. Return the **final morphed frame**.  

#### **Output:**  
A **NumPy array** representing the **morphed frame** with blended image features.  

In [ ]:
def generate_morphed_frame(img1, img2, points1, points2, morphed_points, triangle_indices, alpha):
    """
    Generates a single morphed frame by applying warping transformations on triangular regions.
    """
    blended_image = np.zeros_like(img1)

    for t in triangle_indices:
        # Define corresponding triangles in both input images and the interpolated result
        tri_img1 = np.array([points1[t[0]], points1[t[1]], points1[t[2]]], dtype=np.int32)
        tri_img2 = np.array([points2[t[0]], points2[t[1]], points2[t[2]]], dtype=np.int32)
        tri_morphed = np.array([morphed_points[t[0]], morphed_points[t[1]], morphed_points[t[2]]], dtype=np.int32)

        # Apply transformation and blend the triangle into the output frame
        compute_morphed_triangle(img1, img2, tri_img1, tri_img2, tri_morphed, alpha, blended_image)

    return cv2.medianBlur(blended_image, 5)

### **Function: `create_morph_gif(img1, img2, points1, points2, filename='output1.gif')`**  
#### **Purpose:**  
Generates a **morphing animation (GIF)** between two face images using **Delaunay triangulation** and **image warping**.  

#### **Key Steps:**  
1. **Compute Delaunay triangulation** using `delaunay_triangulation(img1, points1)`.  
2. **Initialize frames list** to store morphed images.  
3. **Iterate over 50 steps (α ∈ [0,1])** ,i.e. 50 frames, to gradually blend `img1` and `img2`:  
   - Compute **intermediate landmark points** using `generate_morphed_points()`.  
   - Generate a **morphed frame** using `generate_morphed_frame()`.
   - Append the processed frame to the list.  
4. **Save the frames as a GIF** using `imageio.mimsave()`.  

#### **Output:**  
A smooth **face morphing animation** between `img1` and `img2`, saved as `output1.gif`. 🎭✨  


In [ ]:
def create_morph_gif(img1, img2, points1, points2, filename='output1.gif'):
    """
    Generates a smooth transition from img1 to img2 by applying Delaunay triangulation-based morphing.
    The function creates an animated GIF that illustrates the transformation process.
    """
    # Generate triangulation indices for landmark points
    triangle_indices = delaunay_triangulation(img1,points1)

    # List to store individual transition frames
    morph_frames = []

    # Generate intermediate frames with varying blend ratios
    for alpha in np.linspace(0, 1, 50):

        # Compute interpolated landmark positions uding weighted avg
        morphed_points = generate_morphed_points(points1, points2, alpha)

        # generate morphed frame
        morph_frames.append(generate_morphed_frame(img1, img2, points1, points2, morphed_points, triangle_indices, alpha))

    # Export the generated frames as an animated GIF
    imageio.mimsave(filename, morph_frames, fps=50)

Loading  Image input

In [ ]:
# Load an image from file
img1 = cv2.imread("image1.jpg")
img2 = cv2.imread("image2.jpg")

# Validate image loading
if img1 is None or img2 is None:
    raise FileNotFoundError("One or both images could not be loaded. Please check the file paths.")

img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

Part A - TIe pts giveen

Reading tie points from text file and storing in lists

In [ ]:
points1 = []
points2 = []

with open("points.txt") as f:
    num_of_tie_points = int(f.readline())
    for _ in range(num_of_tie_points):
        # Read each line of tie point coordinates and add them to their respective lists
        x1, y1, x2, y2 = [int(i) for i in f.readline().split()]

        points1.append((x1, y1))
        points2.append((x2, y2))

# Add the corner points to the list of points for the first image
points1.append((0, 0))
points1.append((img1.shape[1] - 1, 0))
points1.append((0, img1.shape[0] - 1))
points1.append((img1.shape[1] - 1, img1.shape[0] - 1))

# Add the corner points to the list of points for the second image
points2.append((0, 0))
points2.append((img2.shape[1] - 1, 0))
points2.append((0, img2.shape[0] - 1))
points2.append((img2.shape[1] - 1, img2.shape[0] - 1))

In [ ]:
create_morph_gif(img1, img2, points1, points2, "output1.gif")

In [ ]:
# Display the GIF
Image(filename='output1.gif')

Part B- Ties pts not given

In [ ]:
# Get landmarks for both images
landmarks1 = get_landmarks(img1)
landmarks2 = get_landmarks(img2)

if not landmarks1 or not landmarks2:
    raise ValueError("Landmarks could not be detected in one or both images. Please check the images.")

# Convert landmarks to arrays
points1 = np.array(landmarks1)
points2 = np.array(landmarks2)

points1_list = [(int(pt[0]), int(pt[1])) for pt in points1]
points2_list = [(int(pt[0]), int(pt[1])) for pt in points2]

# Print the corresponding tie points
print("Points from image 1:", points1_list)
print("Points from image 2:", points2_list)

In [ ]:
create_morph_gif(img1, img2, points1_list, points2_list, "output2.gif")

In [ ]:
# Display the GIF
Image(filename='output2.gif')


### **Image morphing from human to animal by manually adding tie points**

Loading human and tiger picture

In [ ]:
# Load an image from file
img1 = cv2.imread("human.jpg")
img2 = cv2.imread("tiger1.jpg")

# Validate image loading
if img1 is None or img2 is None:
    raise FileNotFoundError("One or both images could not be loaded. Please check the file paths.")

img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

Performing morphing on human and tiger image by manually marking tie points

In [ ]:
points1 = []
points2 = []

with open("points2.txt") as f:
    num_of_tie_points = int(f.readline())
    for _ in range(num_of_tie_points):
        # Read each line of tie point coordinates and add them to their respective lists
        x1, y1, x2, y2 = [int(i) for i in f.readline().split()]

        points1.append((x1, y1))
        points2.append((x2, y2))

# Add the corner points to the list of points for the first image
points1.append((0, 0))
points1.append((img1.shape[1] - 1, 0))
points1.append((0, img1.shape[0] - 1))
points1.append((img1.shape[1] - 1, img1.shape[0] - 1))

# Add the corner points to the list of points for the second image
points2.append((0, 0))
points2.append((img2.shape[1] - 1, 0))
points2.append((0, img2.shape[0] - 1))
points2.append((img2.shape[1] - 1, img2.shape[0] - 1))

Performing image morphing showing transition from human to tiger

In [ ]:
create_morph_gif(img1, img2, points1, points2, "output.gif")

In [ ]:
# Display the GIF
Image(filename='output.gif')

### **Methodology Used**  

1. **Delaunay Triangulation:**  
   - Generate **triangular meshes** over the face using **Delaunay triangulation**, ensuring consistent triangular regions for both images.  

2. **Landmark Point Interpolation:**  
   - Compute **intermediate feature points** by **linearly interpolating** between corresponding points in `img1` and `img2` for different blending ratios (α ∈ [0,1]).  

3. **Triangle Warping & Blending:**  
   - For each triangle, apply **affine transformation** to warp corresponding regions from both images.  
   - Blend the transformed regions proportionally based on α.  

4. **Frame Generation:**  
   - Repeat the process over 50 steps, storing intermediate **morphed frames**.  
   - Apply **median filtering** to smooth transitions.  

5. **GIF Creation:**  
   - Combine all frames into a **GIF animation** using `imageio.mimsave()`.  

For tie points generation, I have used frontal face detector of dlib library which detects  68 facial points. Additionally, I have added the corner points.